# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [22]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [23]:
# Load in file
city_file = "../WeatherPy/city_data.csv"
city_file_df = pd.read_csv(city_file)
city_file_df.head()

,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kumluca,36.37,30.29,31.6,41,0,4.04,TR,1594396904
1,Narsaq,60.92,-46.05,9.0,93,75,1.50,GL,1594396907
2,Kapaa,22.08,-159.32,26.0,78,90,7.20,US,1594396909
3,Arraial do Cabo,-22.97,-42.03,23.0,56,75,3.10,BR,1594396912
4,Kajaani,64.23,27.73,13.0,76,100,4.10,FI,1594396913


In [24]:
#Convert celsius to farenheit
#Fahrenheit = (Celsius * 9/5) + 32
city_file_df["Max Temp"] = city_file_df["Max Temp"] * 9/5 + 32
city_file_df

,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kumluca,36.37,30.29,88.880,41,0,4.04,TR,1594396904
1,Narsaq,60.92,-46.05,48.200,93,75,1.50,GL,1594396907
2,Kapaa,22.08,-159.32,78.800,78,90,7.20,US,1594396909
3,Arraial do Cabo,-22.97,-42.03,73.400,56,75,3.10,BR,1594396912
4,Kajaani,64.23,27.73,55.400,76,100,4.10,FI,1594396913
...,...,...,...,...,...,...,...,...,...
544,Bubaque,11.28,-15.83,81.662,80,99,6.49,GW,1594397584
545,Benemérito de las Américas,16.52,-90.64,82.400,79,59,1.17,MX,1594397584
546,Clyde River,70.47,-68.59,60.800,44,75,2.60,CA,1594397584
547,Shaowu,27.34,117.48,74.678,90,100,1.22,CN,1594397585


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [26]:
# Store latitude and longitude in locations
locations = city_file_df[["Latitude", "Longitude"]]

# Store humidity
humidity = city_file_df["Humidity"]

# Plot Heatmap
figure_layout = layout={
        'width': '800px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'
}

#fig = gmaps.figure()
fig = gmaps.figure(map_type="HYBRID", zoom_level=2, center=(20,30), layout = figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Save figure
plt.savefig("humidity_info.png")

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='800px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
#Drop null values
print(f"Total number of cities: {len(city_file_df)}")
print("-------------------")
city_file_clean = city_file_df.dropna()
print(f"Total number of cities without null values: {len(city_file_clean)}")
print("-------------------")
city_file_clean.head()

Total number of cities: 549
-------------------
Total number of cities without null values: 544
-------------------


,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kumluca,36.37,30.29,88.88,41,0,4.04,TR,1594396904
1,Narsaq,60.92,-46.05,48.20,93,75,1.50,GL,1594396907
2,Kapaa,22.08,-159.32,78.80,78,90,7.20,US,1594396909
3,Arraial do Cabo,-22.97,-42.03,73.40,56,75,3.10,BR,1594396912
4,Kajaani,64.23,27.73,55.40,76,100,4.10,FI,1594396913


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
# Make a copy of the cleaned city file instead of creating a variable that calls the original dataframe
new_city_file_clean = city_file_clean.copy()
new_city_file_clean.head()

,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kumluca,36.37,30.29,88.88,41,0,4.04,TR,1594396904
1,Narsaq,60.92,-46.05,48.20,93,75,1.50,GL,1594396907
2,Kapaa,22.08,-159.32,78.80,78,90,7.20,US,1594396909
3,Arraial do Cabo,-22.97,-42.03,73.40,56,75,3.10,BR,1594396912
4,Kajaani,64.23,27.73,55.40,76,100,4.10,FI,1594396913


In [29]:
new_city_file_clean["Hotel Name"] = ""
new_city_file_clean.head()

,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Kumluca,36.37,30.29,88.88,41,0,4.04,TR,1594396904,
1,Narsaq,60.92,-46.05,48.20,93,75,1.50,GL,1594396907,
2,Kapaa,22.08,-159.32,78.80,78,90,7.20,US,1594396909,
3,Arraial do Cabo,-22.97,-42.03,73.40,56,75,3.10,BR,1594396912,
4,Kajaani,64.23,27.73,55.40,76,100,4.10,FI,1594396913,


In [30]:
# Grab city names
city_name = new_city_file_clean.loc[:,"City Name"]
city_name

0                         Kumluca
1                          Narsaq
2                           Kapaa
3                 Arraial do Cabo
4                         Kajaani
                  ...            
544                       Bubaque
545    Benemérito de las Américas
546                   Clyde River
547                        Shaowu
548                   Santa Marta
Name: City Name, Length: 544, dtype: object

In [31]:
hotel_places = new_city_file_clean.loc[(new_city_file_clean["Max Temp"] > 70) &
                              (new_city_file_clean["Max Temp"] < 80) &
                              (new_city_file_clean["Wind Speed"] < 10) &
                              (new_city_file_clean["Cloudiness"] == 0)]

hotels_df = hotel_places.copy()
hotels_df

,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
56,Phalaborwa,-23.94,31.14,78.800,47,0,4.60,ZA,1594397009,
137,Ponta do Sol,32.67,-17.10,75.992,68,0,2.61,PT,1594397075,
138,Kropotkin,45.44,40.58,75.002,59,0,2.90,RU,1594397076,
165,Kastamonu Province,41.50,33.67,73.400,43,0,6.20,TR,1594397091,
221,Belaya,49.03,25.77,76.766,76,0,3.31,UA,1594397175,
236,Tatarbunary,45.84,29.61,77.648,41,0,4.68,UA,1594397187,
271,Nsanje,-16.92,35.26,75.182,63,0,2.06,MW,1594397215,
281,Benguela,-12.58,13.41,75.290,62,0,4.78,AO,1594397226,
288,Polovinnoye,54.79,65.99,76.010,78,0,2.17,RU,1594397231,
318,Richards Bay,-28.78,32.04,78.152,38,0,6.62,ZA,1594397249,


In [32]:
"""
From Googles Maps Places API
https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY

parameters: location, radius, type, keyword, key
"""

# geocoordinates
target_radius = 5000
target_type = "lodging"
target_keyword = "hotel"

# set up a parameters dictionary
params = {
    "radius": target_radius,
    "type": target_type,
    "keyword": target_keyword,
    "key": g_key
}

counter = 1

for index, row in hotels_df.iterrows():
    
        # base url
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
        #get the longitude and latitude of each city
        lat = row["Latitude"]
        long = row["Longitude"]
        city_name = row["City Name"]
        
        #add location to params dictionary 
        params["location"] = f"{lat},{long}"
        
        #assemble url and make api request
        # run a request using params dictionary
        response = requests.get(base_url, params=params).json()
        #print(json.dumps(response, indent=4, sort_keys=True))
        
        # extract results
        results = response['results']
        
        try:
            print(f"Closest hotel to {city_name} is {results[0]['name']}.")
        
            hotels_df.loc[index, 'Hotel Name'] = results[0]['name']
        
        except (KeyError, IndexError):
        
            print("Missing field/result... skipping.")
        
print("------------")

Closest hotel to Phalaborwa is Sefapane Lodge & Safaris.
Closest hotel to Ponta do Sol is Estalagem Da Ponta Do Sol.
Closest hotel to Kropotkin is Luxury House ECO-hotel & Spa.
Missing field/result... skipping.
Closest hotel to Belaya is Приватний гуртожиток.
Missing field/result... skipping.
Closest hotel to Nsanje is Shumba Lodge.
Closest hotel to Benguela is Aparthotel Mil Cidades.
Missing field/result... skipping.
Closest hotel to Richards Bay is BON Hotel Waterfront Richards Bay.
Closest hotel to Gimli is Lakeview Gimli Resort.
Missing field/result... skipping.
Closest hotel to Sines Municipality is Hotel Sinerama.
Closest hotel to Inhambane is Hotel Casa Do Capitão.
Missing field/result... skipping.
Closest hotel to Mikhaylovka is Mini-hotel Meduza.
------------


In [35]:
hotels_df

,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
56,Phalaborwa,-23.94,31.14,78.800,47,0,4.60,ZA,1594397009,Sefapane Lodge & Safaris
137,Ponta do Sol,32.67,-17.10,75.992,68,0,2.61,PT,1594397075,Estalagem Da Ponta Do Sol
138,Kropotkin,45.44,40.58,75.002,59,0,2.90,RU,1594397076,Luxury House ECO-hotel & Spa
165,Kastamonu Province,41.50,33.67,73.400,43,0,6.20,TR,1594397091,
221,Belaya,49.03,25.77,76.766,76,0,3.31,UA,1594397175,Приватний гуртожиток
236,Tatarbunary,45.84,29.61,77.648,41,0,4.68,UA,1594397187,
271,Nsanje,-16.92,35.26,75.182,63,0,2.06,MW,1594397215,Shumba Lodge
281,Benguela,-12.58,13.41,75.290,62,0,4.78,AO,1594397226,Aparthotel Mil Cidades
288,Polovinnoye,54.79,65.99,76.010,78,0,2.17,RU,1594397231,
318,Richards Bay,-28.78,32.04,78.152,38,0,6.62,ZA,1594397249,BON Hotel Waterfront Richards Bay


In [36]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotels_df[["Latitude", "Longitude"]]

In [38]:
# Add marker layer ontop of heat map
# Display figure
fig = gmaps.figure(map_type="HYBRID", zoom_level=2, center=(20,30), layout = figure_layout)

figure_layout = layout={
        'width': '800px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'
}

# add markers and layers
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig.add_layer(heat_layer)
plt.savefig("hotel_info.png")

# show figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='800px'))

<Figure size 432x288 with 0 Axes>